### OBJETIVO: TRANSFORMAR ESTE NOTEBOOK NUM SCRIPT QUE POSSA SER USADO PARA QUALQUER REDE

In [2]:
import pandas as pd
from igraph import Graph
import metrics_functions
import importlib
import numpy as np
from IPython.display import display

# Default modules RWR

## Load data

In [5]:
reactome_modules = pd.read_csv(
    "../../data/processed/reactome_modules_apid_huri.csv",
    converters={'protein_id': eval, 'protein_index': eval}
    )
print(reactome_modules.shape[0])
display(reactome_modules.head(2))

disgenet_sca_modules = pd.read_csv(
    "../../data/processed/disgenet_sca_modules_apid_huri.csv",
    converters={'protein_id': eval, 'protein_index': eval}
    )
print(disgenet_sca_modules.shape[0])
display(disgenet_sca_modules.head(2))

disgenet_conservative_modules = pd.read_csv(
    "../../data/processed/disgenet_conservative_modules_apid_huri.csv",
    converters={'protein_id': eval, 'protein_index': eval}
    )
print(disgenet_conservative_modules.shape[0])
display(disgenet_conservative_modules.head(2))

426


,module_id,protein_id,protein_index,module_size
0,R-HSA-1031716,"[TRIM10, TRIM22, IRF9, IFI30, TRIM38, TRIM3, T...","[10480, 6234, 7001, 11737, 4192, 8083, 7685, 8...",73
1,R-HSA-112379,"[CDK7, CDK9, SUPT16H, LEO1, ERCC2, ERCC3, RTF1...","[3063, 2790, 512, 5493, 2565, 2775, 5494, 4148...",56


204


,module_id,protein_id,protein_index,module_size
0,C0001418,"[ABL1, ALOX5, ALOX12B, APC, BIRC5, APOA1, APOE...","[2510, 3877, 10381, 853, 2600, 2518, 2195, 789...",134
1,C0001973,"[NAT1, ABO, ADCY5, ADCY7, ADH1A, ADH1B, ADH1C,...","[14914, 16411, 13350, 9792, 16667, 9783, 14841...",325


300


,module_id,protein_id,protein_index,module_size
0,C0000786,"[IFI44, MMP12, CD226, CEACAM6, MMP15, FLRT3, I...","[12291, 13317, 9737, 9739, 1548, 11791, 16923,...",102
1,C0000822,"[IFI44, MMP12, CD226, CEACAM6, MMP15, FLRT3, I...","[12291, 13317, 9737, 9739, 1548, 11791, 16923,...",102


In [4]:
apid_huri_proteins = pd.read_csv("../../data/processed/apid_huri_gene_ids.csv", sep=',', header=0)
graph_apid_huri = Graph.Read_GML("../../data/processed/apid_huri_graph")
apid_huri_adj_matrix = np.load("../../data/processed/apid_huri_adjacency_matrix.npy")

## Reactome Modules

In [7]:
importlib.reload(metrics_functions)
reactome_metrics = metrics_functions.random_walk_restart(
    graph_apid_huri, reactome_modules, n_jobs=4
    )
reactome_metrics.to_csv('../../data/processed/metrics/reactome_metrics_apid_huri.csv')
reactome_metrics.head()

  0%|          | 0/426 [00:00<?, ?it/s]

module_id,R-HSA-1031716,R-HSA-112379,R-HSA-112385,R-HSA-1168640,R-HSA-1214188,R-HSA-1234159,R-HSA-141409,R-HSA-141422,R-HSA-141431,R-HSA-141439,...,R-HSA-9670149,R-HSA-9698928,R-HSA-9710490,R-HSA-977224,R-HSA-983140,R-HSA-983147,R-HSA-983156,R-HSA-983157,R-HSA-983259,R-HSA-983266
SCRIB,0.000163,0.000143,0.000143,0.000144,0.000126,0.000518,0.000152,0.000152,0.000152,0.000152,...,0.000140,0.000134,0.000127,0.000126,0.000183,0.000187,0.000183,0.000183,0.000166,0.000166
ARHGEF7,0.000101,0.000054,0.000054,0.000063,0.000046,0.000066,0.000060,0.000060,0.000060,0.000060,...,0.000049,0.000046,0.000046,0.000046,0.000096,0.000102,0.000096,0.000096,0.000129,0.000129
NET1,0.000018,0.000015,0.000015,0.000014,0.000014,0.000018,0.000017,0.000017,0.000017,0.000017,...,0.000014,0.000014,0.000013,0.000013,0.000018,0.000018,0.000018,0.000018,0.000020,0.000020
KCNA5,0.000057,0.000036,0.000036,0.000037,0.000065,0.000041,0.000041,0.000041,0.000041,0.000041,...,0.000048,0.000066,0.000063,0.000067,0.000049,0.000049,0.000049,0.000049,0.000052,0.000052
VWCE,0.000210,0.000068,0.000068,0.000066,0.000057,0.000071,0.000068,0.000068,0.000068,0.000068,...,0.000064,0.000059,0.000059,0.000057,0.000114,0.000123,0.000114,0.000114,0.000072,0.000072


In [8]:
# Labels
reactome_labels = np.zeros(reactome_metrics.shape)
column = 0
for indexes in reactome_modules['protein_index'].values:
    for pos in indexes:
        reactome_labels[pos][column] = 1
    column += 1
np.savetxt("../../data/processed/reactome_labels_apid_huri.csv", reactome_labels, delimiter=",")

## Disgenet SCA Modules

In [9]:
importlib.reload(metrics_functions)
disease_sca_metrics = metrics_functions.random_walk_restart(graph_apid_huri, disgenet_sca_modules)

disease_sca_metrics.to_csv('../../data/processed/metrics/disgenet_sca_apid_huri.csv')
disease_sca_metrics.head()

  0%|          | 0/204 [00:00<?, ?it/s]

module_id,C0001418,C0001973,C0002152,C0002395,C0002736,C0003873,C0004153,C0004238,C0004352,C0005684,...,C3714758,C4277682,C4277690,C4279912,C4505456,C4552091,C4704862,C4707243,C4721507,C4722327
SCRIB,0.000177,0.000292,0.000162,0.000200,0.000153,0.000220,0.000159,0.000353,0.000227,0.000176,...,0.000201,0.000179,0.000182,0.000179,0.000152,0.000201,0.000201,0.000146,0.000216,0.000172
ARHGEF7,0.000065,0.000083,0.000087,0.000070,0.000059,0.000064,0.000120,0.000093,0.000105,0.000102,...,0.000066,0.000058,0.000083,0.000058,0.000076,0.000066,0.000066,0.000067,0.000060,0.000059
NET1,0.000017,0.000027,0.000016,0.000016,0.000020,0.000018,0.000016,0.000022,0.000054,0.000024,...,0.000018,0.000033,0.000020,0.000033,0.000054,0.000018,0.000018,0.000016,0.000096,0.000019
KCNA5,0.000052,0.000086,0.000109,0.000116,0.000049,0.000059,0.000077,0.000913,0.000075,0.000054,...,0.000067,0.000060,0.000074,0.000060,0.000050,0.000067,0.000067,0.000052,0.000061,0.000061
VWCE,0.000087,0.000080,0.000084,0.000109,0.000101,0.000118,0.000087,0.000080,0.000084,0.000077,...,0.000136,0.000077,0.000077,0.000077,0.000076,0.000136,0.000136,0.000278,0.000095,0.000076


In [11]:
disgenet_sca_labels = np.zeros(disease_sca_metrics.shape)
column = 0
for indexes in disgenet_sca_modules['protein_index'].values:
    for pos in indexes:
        disgenet_sca_labels[pos][column] = 1
    column += 1
np.savetxt("../../data/processed/disgenet_sca_labels_apid_huri.csv", disgenet_sca_labels, delimiter=",")

## Disgenet Conservative Modules

In [12]:
importlib.reload(metrics_functions)
disease_conservative_metrics = metrics_functions.random_walk_restart(graph_apid_huri, disgenet_conservative_modules)

disease_conservative_metrics.to_csv('../../data/processed/metrics/disgenet_conservative_apid_huri.csv')
disease_conservative_metrics.head()

  0%|          | 0/300 [00:00<?, ?it/s]

module_id,C0000786,C0000822,C0001418,C0001787,C0001973,C0002152,C0002395,C0002736,C0003873,C0004096,...,C4317123,C4505436,C4505456,C4552091,C4552766,C4704862,C4707243,C4721453,C4721507,C4722327
SCRIB,0.000149,0.000149,0.000181,0.000149,0.000317,0.000164,0.000185,0.000152,0.000233,0.000152,...,0.000220,0.000222,0.000152,0.000212,0.000149,0.000212,0.000145,0.000171,0.000236,0.000177
ARHGEF7,0.000068,0.000068,0.000063,0.000057,0.000075,0.000094,0.000074,0.000058,0.000064,0.000097,...,0.000072,0.000072,0.000075,0.000066,0.000068,0.000066,0.000061,0.000123,0.000061,0.000060
NET1,0.000015,0.000015,0.000017,0.000015,0.000025,0.000016,0.000016,0.000021,0.000017,0.000050,...,0.000024,0.000024,0.000057,0.000019,0.000015,0.000019,0.000016,0.000018,0.000064,0.000020
KCNA5,0.000065,0.000065,0.000051,0.000047,0.000089,0.000105,0.000133,0.000050,0.000057,0.000052,...,0.000222,0.000223,0.000051,0.000058,0.000065,0.000058,0.000049,0.000057,0.000057,0.000063
VWCE,0.000182,0.000182,0.000083,0.000091,0.000082,0.000089,0.000120,0.000110,0.000130,0.000261,...,0.000075,0.000075,0.000078,0.000155,0.000182,0.000155,0.000336,0.000098,0.000089,0.000070


In [13]:
disgenet_conservative_labels = np.zeros(disease_conservative_metrics.shape)
column = 0
for indexes in disgenet_conservative_modules['protein_index'].values:
    for pos in indexes:
        disgenet_conservative_labels[pos][column] = 1
    column += 1
np.savetxt("../../data/processed/disgenet_conservative_labels_apid_huri.csv", disgenet_conservative_labels, delimiter=",")

# Modules with false annotations

## Load data

In [14]:
reactome_modules = pd.read_csv(
    "../../data/processed/reactome_modules_fa_apid_huri.csv",
    converters={'protein_id': eval, 'protein_index': eval}
    )
print(reactome_modules.shape[0])
display(reactome_modules.head(2))

disgenet_sca_modules = pd.read_csv(
    "../../data/processed/disgenet_sca_modules_fa_apid_huri.csv",
    converters={'protein_id': eval, 'protein_index': eval}
    )
print(disgenet_sca_modules.shape[0])
display(disgenet_sca_modules.head(2))

disgenet_conservative_modules = pd.read_csv(
    "../../data/processed/disgenet_conservative_modules_fa_apid_huri.csv",
    converters={'protein_id': eval, 'protein_index': eval}
    )
print(disgenet_conservative_modules.shape[0])
display(disgenet_conservative_modules.head(2))

426


,module_id,protein_id,protein_index
0,R-HSA-1031716,"[TRIM10, TRIM22, IRF9, IFI30, TRIM38, TRIM3, T...","[10480, 6234, 7001, 11737, 4192, 8083, 7685, 8..."
1,R-HSA-112379,"[CDK7, CDK9, SUPT16H, LEO1, ERCC2, ERCC3, RTF1...","[3063, 2790, 512, 5493, 2565, 2775, 5494, 4148..."


204


,module_id,protein_id,protein_index
0,C0001418,"[ABL1, ALOX5, ALOX12B, APC, BIRC5, APOA1, APOE...","[2510, 3877, 10381, 853, 2600, 2518, 2195, 789..."
1,C0001973,"[NAT1, ABO, ADCY5, ADCY7, ADH1A, ADH1B, ADH1C,...","[14914, 16411, 13350, 9792, 16667, 9783, 14841..."


300


,module_id,protein_id,protein_index
0,C0000786,"[IFI44, MMP12, CD226, CEACAM6, MMP15, FLRT3, I...","[12291, 13317, 9737, 9739, 1548, 11791, 16923,..."
1,C0000822,"[IFI44, MMP12, CD226, CEACAM6, MMP15, FLRT3, I...","[12291, 13317, 9737, 9739, 1548, 11791, 16923,..."


In [15]:
apid_huri_proteins = pd.read_csv("../../data/processed/apid_huri_gene_ids.csv", sep=',', header=0)
graph_apid_huri = Graph.Read_GML("../../data/processed/apid_huri_graph")
apid_huri_adj_matrix = np.load("../../data/processed/apid_huri_adjacency_matrix.npy")

## Reactome Modules

In [16]:
importlib.reload(metrics_functions)
reactome_metrics = metrics_functions.random_walk_restart(
    graph_apid_huri, reactome_modules, n_jobs=4
    )
reactome_metrics.to_csv('../../data/processed/metrics/reactome_fa_metrics_apid_huri.csv')
reactome_metrics.head()

  0%|          | 0/426 [00:00<?, ?it/s]

module_id,R-HSA-1031716,R-HSA-112379,R-HSA-112385,R-HSA-1168640,R-HSA-1214188,R-HSA-1234159,R-HSA-141409,R-HSA-141422,R-HSA-141431,R-HSA-141439,...,R-HSA-9670149,R-HSA-9698928,R-HSA-9710490,R-HSA-977224,R-HSA-983140,R-HSA-983147,R-HSA-983156,R-HSA-983157,R-HSA-983259,R-HSA-983266
SCRIB,0.000161,0.000143,0.000143,0.000143,0.000140,0.000485,0.000154,0.000154,0.000154,0.000154,...,0.000142,0.000134,0.000170,0.000127,0.000209,0.000184,0.000209,0.000209,0.000165,0.000165
ARHGEF7,0.000096,0.000053,0.000053,0.000064,0.000046,0.000066,0.000061,0.000061,0.000061,0.000061,...,0.000052,0.000047,0.000128,0.000047,0.000094,0.000098,0.000094,0.000094,0.000125,0.000125
NET1,0.000017,0.000015,0.000015,0.000015,0.000014,0.000017,0.000017,0.000017,0.000017,0.000017,...,0.000014,0.000014,0.000014,0.000013,0.000018,0.000017,0.000018,0.000018,0.000020,0.000020
KCNA5,0.000056,0.000037,0.000037,0.000038,0.000064,0.000042,0.000049,0.000049,0.000049,0.000049,...,0.000048,0.000064,0.000061,0.000064,0.000051,0.000049,0.000051,0.000051,0.000051,0.000051
VWCE,0.000201,0.000068,0.000068,0.000067,0.000058,0.000071,0.000068,0.000068,0.000068,0.000068,...,0.000065,0.000063,0.000062,0.000058,0.000113,0.000118,0.000113,0.000113,0.000071,0.000071


In [17]:
# Labels
reactome_labels = np.zeros(reactome_metrics.shape)
column = 0
for indexes in reactome_modules['protein_index'].values:
    for pos in indexes:
        reactome_labels[pos][column] = 1
    column += 1
np.savetxt("../../data/processed/reactome_fa_labels_apid_huri.csv", reactome_labels, delimiter=",")

## Disgenet SCA Modules

In [18]:
importlib.reload(metrics_functions)
disease_sca_metrics = metrics_functions.random_walk_restart(graph_apid_huri, disgenet_sca_modules)

disease_sca_metrics.to_csv('../../data/processed/metrics/disgenet_sca_fa_apid_huri.csv')
disease_sca_metrics.head()

  0%|          | 0/204 [00:00<?, ?it/s]

module_id,C0001418,C0001973,C0002152,C0002395,C0002736,C0003873,C0004153,C0004238,C0004352,C0005684,...,C3714758,C4277682,C4277690,C4279912,C4505456,C4552091,C4704862,C4707243,C4721507,C4722327
SCRIB,0.000262,0.000280,0.000161,0.000200,0.000160,0.000215,0.000157,0.000335,0.000220,0.000179,...,0.000198,0.000179,0.000179,0.000179,0.000151,0.000198,0.000198,0.000145,0.000210,0.000170
ARHGEF7,0.000065,0.000082,0.000084,0.000069,0.000227,0.000063,0.000115,0.000090,0.000101,0.000098,...,0.000066,0.000060,0.000081,0.000060,0.000077,0.000066,0.000066,0.000066,0.000059,0.000060
NET1,0.000018,0.000026,0.000016,0.000016,0.000020,0.000018,0.000017,0.000022,0.000051,0.000025,...,0.000018,0.000032,0.000019,0.000032,0.000051,0.000018,0.000018,0.000016,0.000089,0.000019
KCNA5,0.000053,0.000083,0.000107,0.000117,0.000055,0.000079,0.000074,0.000837,0.000073,0.000055,...,0.000067,0.000062,0.000072,0.000062,0.000049,0.000067,0.000067,0.000051,0.000060,0.000059
VWCE,0.000086,0.000100,0.000086,0.000106,0.000099,0.000152,0.000085,0.000081,0.000086,0.000080,...,0.000135,0.000078,0.000080,0.000078,0.000076,0.000135,0.000135,0.000261,0.000098,0.000076


In [19]:
disgenet_sca_labels = np.zeros(disease_sca_metrics.shape)
column = 0
for indexes in disgenet_sca_modules['protein_index'].values:
    for pos in indexes:
        disgenet_sca_labels[pos][column] = 1
    column += 1
np.savetxt("../../data/processed/disgenet_sca_fa_labels_apid_huri.csv", disgenet_sca_labels, delimiter=",")

## Disgenet Conservative Modules

In [20]:
importlib.reload(metrics_functions)
disease_conservative_metrics = metrics_functions.random_walk_restart(graph_apid_huri, disgenet_conservative_modules)

disease_conservative_metrics.to_csv('../../data/processed/metrics/disgenet_conservative_fa_apid_huri.csv')
disease_conservative_metrics.head()

  0%|          | 0/300 [00:00<?, ?it/s]

module_id,C0000786,C0000822,C0001418,C0001787,C0001973,C0002152,C0002395,C0002736,C0003873,C0004096,...,C4317123,C4505436,C4505456,C4552091,C4552766,C4704862,C4707243,C4721453,C4721507,C4722327
SCRIB,0.000149,0.000149,0.000180,0.000148,0.000302,0.000164,0.000181,0.000152,0.000232,0.000152,...,0.000212,0.000218,0.000155,0.000206,0.000149,0.000206,0.000146,0.000169,0.000231,0.000177
ARHGEF7,0.000067,0.000067,0.000064,0.000057,0.000074,0.000090,0.000073,0.000057,0.000066,0.000102,...,0.000071,0.000071,0.000074,0.000065,0.000067,0.000065,0.000061,0.000119,0.000060,0.000060
NET1,0.000016,0.000016,0.000016,0.000015,0.000024,0.000016,0.000016,0.000020,0.000017,0.000055,...,0.000023,0.000023,0.000063,0.000019,0.000016,0.000019,0.000016,0.000018,0.000060,0.000019
KCNA5,0.000064,0.000064,0.000051,0.000047,0.000085,0.000101,0.000126,0.000050,0.000056,0.000052,...,0.000212,0.000213,0.000051,0.000061,0.000064,0.000061,0.000049,0.000056,0.000057,0.000062
VWCE,0.000174,0.000174,0.000081,0.000089,0.000080,0.000087,0.000121,0.000112,0.000129,0.000246,...,0.000078,0.000076,0.000078,0.000149,0.000174,0.000149,0.000315,0.000096,0.000089,0.000078


In [21]:
disgenet_conservative_labels = np.zeros(disease_conservative_metrics.shape)
column = 0
for indexes in disgenet_conservative_modules['protein_index'].values:
    for pos in indexes:
        disgenet_conservative_labels[pos][column] = 1
    column += 1
np.savetxt("../../data/processed/disgenet_conservative_fa_labels_apid_huri.csv", disgenet_conservative_labels, delimiter=",")